In [15]:
import time
import requests
import pandas as pd
import PySimpleGUI as sg

from typing import List
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC

THEME = 'DarkBlack'
HEADLESS = False
AREA_LIST = ['W', 'R', 'C']

sg.theme(THEME)

# Create a new instance of the Chrome driver
chrome_options = webdriver.ChromeOptions()

if HEADLESS:
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--ignore-certificate-errors-spki-list')
    chrome_options.add_argument('--ignore-ssl-errors')

In [16]:
BASE_URL = 'https://miamidade.realforeclose.com/index.cfm'
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options= chrome_options)
driver.get(BASE_URL)

C:\Users\jksoj\AppData\Local\Temp\ipykernel_9264\801500143.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options= chrome_options)
C:\Users\jksoj\AppData\Local\Temp\ipykernel_9264\801500143.py:2: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options= chrome_options)


In [41]:
res = requests.get(BASE_URL)
soup = BeautifulSoup(res.text, "lxml")
website_list = []
for website in soup.find("select", {"id": "JMP_MENU_SEL"}).text.split('\n'):
    if webdriver and website.strip():
        website_list.append(website.strip())
website_list = website_list[1:]

In [17]:
website_list = []
base_select_website = driver.find_element(By.XPATH, '//select[@id="JMP_MENU_SEL"]')
for website in base_select_website.text.split('\n')[1:]:
    if webdriver and website.strip():
        website_list.append(website.strip())


sg.theme(THEME)   # Add a touch of color
# All the stuff inside your window.
layout = [  [sg.Text('Select any of the following website:')],
            [sg.Combo(values=website_list, key='WEBSITE', size=(75, 6))],
            [sg.Button('Ok'), sg.Button('Cancel')] ]

# Create the Window
window = sg.Window('Select Website', layout)
# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()

    if event == sg.WIN_CLOSED or event == 'Cancel' or event == 'Ok': # if user closes window or clicks cancel
        break

window.close()
select_box = Select(base_select_website)
select_box.select_by_visible_text(values["WEBSITE"])
WebDriverWait(driver, 10).until(EC.alert_is_present())
driver.switch_to.alert.accept()
time.sleep(1)
PREFIX = "/".join(driver.current_url.split('/')[:-1])

In [18]:
layout = [  [sg.Text('Select any of the following date:')],
            [sg.Text('Enter Date in YYYY-MM format'), sg.InputText()],
            [sg.Button('Ok'), sg.Button('Cancel')] ]

# Create the Window
window = sg.Window('Getting calender', layout)
# Event Loop to process "events" and get the "values" of the inputs

while True:
    event, values = window.read()
    print('You entered ', values)

    if event == sg.WIN_CLOSED or event == 'Cancel': # if user closes window or clicks cancel
        break
    elif event == 'Ok':
        if len(values[0]) != 7 or '-' not in values[0]:
            print('Enter valid date')
        else:
            break

window.close()
cal_start_date = values[0]+'-01'

You entered  {0: '2022-12'}


In [19]:
cal_url = f'{PREFIX}/index.cfm?zaction=user&zmethod=calendar&selCalDate=%7Bts%20%27{cal_start_date}%2000%3A00%3A00%27%7D'
res = requests.get(cal_url)

In [20]:
soup = BeautifulSoup(res.text, "lxml")
calender = soup.find("div", {"class": "CALDAYBOX"})

In [21]:
final_calender: List[BeautifulSoup] = []
for date in calender.find_all("div"):
    if len(date.text.strip()) > 5:
        final_calender.append(date)

In [22]:
user_section = []
for date in final_calender:
    user_section.append(f'{date.get("dayid")} | {date.text[2:].strip()}')

In [23]:
user_section

['12/01/2022 | Foreclosure0 / 5 FC 11:00 AM ET',
 '12/05/2022 | Foreclosure0 / 3 FC 11:00 AM ET',
 '12/06/2022 | Foreclosure0 / 5 FC 11:00 AM ET',
 '12/07/2022 | Foreclosure0 / 12 FC 11:00 AM ET',
 '12/08/2022 | Foreclosure0 / 5 FC 11:00 AM ET',
 '12/12/2022 | Foreclosure0 / 2 FC 11:00 AM ET',
 '12/13/2022 | Foreclosure3 / 3 FC 11:00 AM ET',
 '12/14/2022 | Foreclosure2 / 4 FC 11:00 AM ET',
 '12/15/2022 | Foreclosure3 / 3 FC 11:00 AM ET',
 '12/16/2022 | Foreclosure0 / 1 FC 11:00 AM ET',
 '12/19/2022 | Foreclosure2 / 2 FC 11:00 AM ET',
 '12/20/2022 | Foreclosure1 / 1 FC 11:00 AM ET',
 '12/21/2022 | Foreclosure1 / 3 FC 11:00 AM ET',
 '12/22/2022 | Foreclosure1 / 1 FC 11:00 AM ET',
 '12/27/2022 | Foreclosure1 / 1 FC 11:00 AM ET',
 '12/28/2022 | Foreclosure1 / 1 FC 11:00 AM ET']

In [24]:
sg.theme(THEME)   # Add a touch of color
# All the stuff inside your window.
layout = [  [sg.Text('Select any of the following date:')],
            [sg.Combo(values=user_section, key='DATE', size=(75, 6))],
            [sg.Button('Ok'), sg.Button('Cancel')] ]

# Create the Window
window = sg.Window('Window Title', layout)
# Event Loop to process "events" and get the "values" of the inputs
while True:
    event, values = window.read()
    print('You entered ', values['DATE'])

    selected_date = values['DATE'].split('|')[0].strip()
    if event == sg.WIN_CLOSED or event == 'Cancel' or event == 'Ok': # if user closes window or clicks cancel
        break

window.close()

You entered  12/07/2022 | Foreclosure0 / 12 FC 11:00 AM ET


In [25]:
BASE_URL_WITH_DATE = f"{PREFIX}/index.cfm?zaction=AUCTION&Zmethod=PREVIEW&AUCTIONDATE={selected_date}"
driver.get(BASE_URL_WITH_DATE)

try:
    email = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH,'//div[@isset="1"]')))
except:
    print('Can not find the auctions.')

In [26]:
all_auctions = []
for area in AREA_LIST:
    try:
        max_pages = int(driver.find_element(By.XPATH, f'//div[@class="Head_{area}"]//span[@id="max{area}A"]').text)
    except:
        print(f'can not get AREA_LIST:{area}')
        continue
    
    print(area, max_pages, f'//div[@class="Head_{area}"]//span[@id="maxCA"]')
    for i in range(max_pages):
        collected_auctions_status = driver.find_elements(By.XPATH, f'//div[@id="Area_{area}"]//div[@isset="1"]//div[@class="AUCTION_STATS"]')
        collected_auctions_details = driver.find_elements(By.XPATH, f'//div[@id="Area_{area}"]//div[@isset="1"]//table[@class="ad_tab"]')

        if collected_auctions_status and collected_auctions_details:
            for status, detail in zip(collected_auctions_status, collected_auctions_details):
                all_auctions.append((status.text, detail.text))

            driver.find_element(By.XPATH, f'//div[@class="Head_{area}"]//div[@class="PageFrame"]//span[@class="PageRight"]/img').click()
            time.sleep(2)

can not get AREA_LIST:W
can not get AREA_LIST:R
C 2 //div[@class="Head_C"]//span[@id="maxCA"]


In [37]:
data:List[dict] = []
for status, detail in all_auctions:
    details = {}

    for key, value in zip(status.split('\n')[::2], status.split('\n')[1::2]):
        details[key] = value.strip()
    
    prev = ""
    for key_value in detail.split('\n'):
        if ":" in key_value:
            key, value = key_value.split(':')
            details[key] = value.strip()
            prev = key
        else:
            details[prev] += " \n" + key_value.strip()
    
    data.append(details)


In [39]:
pd.DataFrame(data).to_excel('out.xlsx')

In [50]:
website_name = PREFIX.split('/')[-1].split('.')[0]
f'{website_name}_{selected_date}'

'duval_12/07/2022'